# Inferencia — High Garden Coffee (CONSISTENTE)
Notebook **auto-contenido y consistente** con tu esquema de datos limpio:

**Se espera un archivo `DATA_CLEAN` con columnas:** `country`, `type`, `year`, `consumption`, `price`, `profit`.

Incluye:
- Splits temporales `rolling-origin`
- Baselines (Naive / SNaive)
- Modelos: Ridge, Lasso, RandomForest (+ XGBoost si está disponible)
- Métricas: RMSE, MAE, sMAPE, MdAPE, MASE
- Intervalos de predicción (Conformal)

👉 **Sin gráficos** (el EDA va aparte).

## 1) Imports, configuración y rutas

In [5]:

import os
import numpy as np
import pandas as pd
from typing import List, Tuple

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import clone
from sklearn.metrics import mean_absolute_error

np.random.seed(42)
pd.set_option("display.max_columns", 200)

# XGBoost opcional
try:
    from xgboost import XGBRegressor
    XGB_AVAILABLE = True
except Exception:
    XGB_AVAILABLE = False

# RUTA al dataset limpio (ajústala si necesitas)
DATA_CLEAN = "data/coffee_clean.csv" # <-- cámbiala si tu archivo se llama distinto
# fallback opcional si existe en /mnt/data
if not os.path.exists(DATA_CLEAN):
    alt = "/mnt/data/coffee_db.csv"
    if os.path.exists(alt):
        DATA_CLEAN = alt  # Úsalo si ya está limpio con las columnas esperadas


## 2) Carga de datos (CONSISTENTE)
Se valida que existan **exactamente** las columnas esperadas. No se hace melt/reshape aquí.

In [6]:

REQUIRED = ["country","type","year","consumption","price","profit"]
df = pd.read_csv(DATA_CLEAN)
# Normaliza nombres por si hay mayúsculas/espacios accidentales
df.columns = [c.strip() for c in df.columns]
missing = [c for c in REQUIRED if c not in df.columns]
assert not missing, f"Faltan columnas requeridas: {missing}. El archivo debe tener {REQUIRED}."
# Tipos y orden
df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
df = df.sort_values(["country","type","year"]).reset_index(drop=True)
FEATURE_BASE = ["country","type","year"]
display(df.head(3))


,country,type,consumption,year,price,revenue,profit,margin,market_share
0,Angola,Robusta/Arabica,1200000,1991,87.686363,1.052236e+08,1.051303e+08,0.999113,0.001025
1,Angola,Robusta/Arabica,1800000,1992,87.686363,1.578355e+08,1.577121e+08,0.999219,0.001483
2,Angola,Robusta/Arabica,2100000,1993,87.686363,1.841414e+08,1.840030e+08,0.999249,0.001671


## 3) Splits temporales (rolling-origin)
Genera pliegues con ventana de entrenamiento creciente y validación de `val_years` años.

In [7]:

def year_based_cv(df, year_col="year", initial_train_years=21, val_years=3, step=1):
    years = sorted([int(y) for y in df[year_col].dropna().unique()])
    if len(years) < (initial_train_years + val_years):
        initial_train_years = max(3, len(years) - val_years - 1)
    for start in range(0, max(1, len(years) - initial_train_years - val_years + 1), step):
        train_years = years[:initial_train_years + start]
        val_years_ = years[initial_train_years + start: initial_train_years + start + val_years]
        tr_idx = df.index[df[year_col].isin(train_years)]
        va_idx = df.index[df[year_col].isin(val_years_)]
        yield tr_idx, va_idx, train_years, val_years_

CV_FOLDS = list(year_based_cv(df, "year", initial_train_years=21, val_years=3, step=1))
len(CV_FOLDS), CV_FOLDS[0][2][-1:], CV_FOLDS[0][3]


(7, [2011], [2012, 2013, 2014])

## 4) Features con *lags* y rolling (sin fuga de información)
Se crean lags/medias móviles por grupo (`country`,`type`).

In [8]:

from typing import Dict

GROUP_COLS = ["country","type"]
TARGETS = {"consumption":"consumption","price":"price","profit":"profit"}

def add_group_safe_lags(df: pd.DataFrame, group_cols: List[str], lag_cols: List[str], 
                        lags=(1,2), add_roll=True) -> pd.DataFrame:
    df = df.copy()
    df = df.sort_values(["year", *group_cols])
    for col in lag_cols:
        base = df.groupby(group_cols, group_keys=False)[col]
        for L in lags:
            df[f"{col}_lag{L}"] = base.shift(L)
        if add_roll:
            s = base.shift(1)  # asegura solo pasado
            for win in [2,3,5]:
                df[f"{col}_ma{win}"] = s.rolling(win, min_periods=1).mean()
                df[f"{col}_std{win}"] = s.rolling(win, min_periods=2).std()
    return df

def build_xy(df: pd.DataFrame, y_col: str):
    # laggeamos: todas numéricas distintas a year + el propio y_col
    num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in ["year"]]
    lag_pool = sorted(set(num_cols + [y_col]))
    df_feat = add_group_safe_lags(df, GROUP_COLS, lag_pool, lags=(1,2), add_roll=True)
    feat_cols = [c for c in df_feat.columns if any(s in c for s in ["_lag", "_ma", "_std"])]
    df_feat["t_index"] = df_feat.groupby(GROUP_COLS).cumcount()
    feat_cols += ["t_index"]
    X = df_feat[feat_cols + GROUP_COLS]
    y = df_feat[y_col]
    return X, y, feat_cols


## 5) Métricas (RMSE, MAE, sMAPE, MdAPE, MASE)

In [9]:

def rmse(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return float(np.sqrt(np.mean((y_true - y_pred)**2)))

def smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true, dtype=float), np.array(y_pred, dtype=float)
    denom = np.abs(y_true) + np.abs(y_pred)
    denom = np.where(denom == 0, 1e-9, denom)
    return float(np.mean(2.0 * np.abs(y_pred - y_true) / denom) * 100)

def mdape(y_true, y_pred):
    y_true, y_pred = np.array(y_true, dtype=float), np.array(y_pred, dtype=float)
    denom = np.where(y_true == 0, 1e-9, np.abs(y_true))
    return float(np.median(np.abs((y_true - y_pred) / denom)) * 100)

def mase(y_true, y_pred, y_train, m=1):
    y_true, y_pred = np.array(y_true, dtype=float), np.array(y_pred, dtype=float)
    y_train = np.array(y_train, dtype=float)
    if len(y_train) <= m:
        return np.nan
    denom = np.mean(np.abs(y_train[m:] - y_train[:-m]))
    denom = denom if denom != 0 else 1e-9
    return float(np.mean(np.abs(y_true - y_pred)) / denom)

def metrics_dict(y_true, y_pred, y_train_for_mase):
    return {
        "RMSE": rmse(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "sMAPE(%)": smape(y_true, y_pred),
        "MdAPE(%)": mdape(y_true, y_pred),
        "MASE": mase(y_true, y_pred, y_train_for_mase, m=1)
    }


## 6) Baselines (Naive / SNaive anual)

In [10]:

def baseline_naive(train_df, val_df, y_col):
    tr = train_df.sort_values(["year", *GROUP_COLS])
    va = val_df.sort_values(["year", *GROUP_COLS])
    last_train = tr.groupby(GROUP_COLS)[y_col].last().to_dict()
    return np.array([ last_train.get((row["country"], row["type"]), tr[y_col].mean())
                       for _, row in va.iterrows() ])

def baseline_snaive(train_df, val_df, y_col, season_m=1):
    # Anual -> m=1 ≈ Naive
    if season_m == 1:
        return baseline_naive(train_df, val_df, y_col)
    return baseline_naive(train_df, val_df, y_col)


## 7) Modelos y preprocesamiento categórico

In [11]:

def make_preprocessor(scale_numeric: bool):
    transformers = [("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), GROUP_COLS)]
    if scale_numeric:
        pre = ColumnTransformer(transformers, remainder="passthrough")
        return Pipeline([("ct", pre), ("scaler", StandardScaler(with_mean=False))])
    else:
        return ColumnTransformer(transformers, remainder="passthrough")

def model_registry():
    models = {
        "ridge": Pipeline([
            ("pre", make_preprocessor(scale_numeric=True)),
            ("mdl", RidgeCV(alphas=np.logspace(-3, 3, 15), cv=5))
        ]),
        "lasso": Pipeline([
            ("pre", make_preprocessor(scale_numeric=True)),
            ("mdl", LassoCV(alphas=np.logspace(-3, 1, 15), cv=5, random_state=42, n_jobs=-1, max_iter=5000))
        ]),
        "rf": Pipeline([
            ("pre", make_preprocessor(scale_numeric=False)),
            ("mdl", RandomForestRegressor(
                n_estimators=500, max_depth=None, min_samples_leaf=2, random_state=42, n_jobs=-1
            ))
        ]),
    }
    if XGB_AVAILABLE:
        models["xgb"] = Pipeline([
            ("pre", make_preprocessor(scale_numeric=False)),
            ("mdl", XGBRegressor(
                n_estimators=800, learning_rate=0.05, max_depth=6, subsample=0.8, colsample_bytree=0.8,
                reg_alpha=0.0, reg_lambda=1.0, random_state=42, tree_method="hist", n_jobs=-1
            ))
        ])
    return models


## 8) Validación cruzada temporal por objetivo

In [12]:

def eval_one_target(df: pd.DataFrame, y_col: str) -> pd.DataFrame:
    X_all, y_all, feat_cols = build_xy(df, y_col)
    rows = []
    models = model_registry()

    # Alinea índices
    X_all = X_all.copy()
    X_all.index = df.index
    y_all = y_all.loc[X_all.index]

    for fold_id, (tr_idx, va_idx, tr_years, va_years) in enumerate(CV_FOLDS, start=1):
        X_tr = X_all.loc[tr_idx].dropna(); y_tr = y_all.loc[X_tr.index]
        X_va = X_all.loc[va_idx].dropna(); y_va = y_all.loc[X_va.index]

        train_df = df.loc[X_tr.index]
        val_df   = df.loc[X_va.index]

        # Baselines
        yhat_naive  = baseline_naive(train_df, val_df, y_col)
        yhat_snaive = baseline_snaive(train_df, val_df, y_col)

        rows.append({"target": y_col, "model": "naive",  "fold": fold_id, **metrics_dict(y_va.values, yhat_naive, y_tr.values)})
        rows.append({"target": y_col, "model": "snaive", "fold": fold_id, **metrics_dict(y_va.values, yhat_snaive, y_tr.values)})

        # Modelos
        for name, pipe in models.items():
            mdl = clone(pipe)
            mdl.fit(X_tr, y_tr)
            pred = mdl.predict(X_va)
            rows.append({"target": y_col, "model": name, "fold": fold_id, **metrics_dict(y_va.values, pred, y_tr.values)})

    return pd.DataFrame(rows)

all_results = []
for logical, y_col in {"consumption":"consumption","price":"price","profit":"profit"}.items():
    print(f"↳ Evaluando {logical} [{y_col}]")
    res = eval_one_target(df, y_col)
    res["logical_target"] = logical
    all_results.append(res)

results_df = pd.concat(all_results, ignore_index=True)
results_df.head()


↳ Evaluando consumption [consumption]


/home/wilson/Documentos/Coffee/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.767e+17, tolerance: 6.105e+13
  model = cd_fast.enet_coordinate_descent_gram(
/home/wilson/Documentos/Coffee/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.399e+17, tolerance: 5.324e+13
  model = cd_fast.enet_coordinate_descent_gram(
/home/wilson/Documentos/Coffee/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the f

↳ Evaluando price [price]


/home/wilson/Documentos/Coffee/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.073e+04, tolerance: 1.324e+02
  model = cd_fast.enet_coordinate_descent_gram(
/home/wilson/Documentos/Coffee/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.151e+02, tolerance: 1.303e+02
  model = cd_fast.enet_coordinate_descent_gram(
/home/wilson/Documentos/Coffee/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the f

↳ Evaluando profit [profit]


/home/wilson/Documentos/Coffee/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.433e+21, tolerance: 1.180e+18
  model = cd_fast.enet_coordinate_descent_gram(
/home/wilson/Documentos/Coffee/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.425e+21, tolerance: 1.180e+18
  model = cd_fast.enet_coordinate_descent_gram(
/home/wilson/Documentos/Coffee/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the f

,target,model,fold,RMSE,MAE,sMAPE(%),MdAPE(%),MASE,logical_target
0,consumption,naive,1,7.075617e+07,4.056620e+07,149.089700,99.882353,14.634080,consumption
1,consumption,snaive,1,7.075617e+07,4.056620e+07,149.089700,99.882353,14.634080,consumption
2,consumption,ridge,1,5.565485e+07,3.013734e+07,129.793788,246.818221,10.871916,consumption
3,consumption,lasso,1,5.736688e+07,3.054859e+07,132.253543,199.842864,11.020273,consumption
4,consumption,rf,1,5.436416e+07,2.614953e+07,113.324583,95.691653,9.433331,consumption


## 9) Resumen por modelo y selección del mejor (según sMAPE)

In [13]:

def summarize_results(df: pd.DataFrame) -> pd.DataFrame:
    agg = df.groupby(["logical_target","model"]).agg(
        RMSE_mean=("RMSE","mean"), MAE_mean=("MAE","mean"),
        sMAPE_mean=("sMAPE(%)","mean"), MdAPE_mean=("MdAPE(%)","mean"),
        MASE_mean=("MASE","mean"), folds=("fold","nunique")
    ).reset_index()
    agg["rank_sMAPE"] = agg.groupby("logical_target")["sMAPE_mean"].rank(method="first")
    return agg.sort_values(["logical_target","rank_sMAPE"])

summary_df = summarize_results(results_df)
summary_df


,logical_target,model,RMSE_mean,MAE_mean,sMAPE_mean,MdAPE_mean,MASE_mean,folds,rank_sMAPE
2,consumption,rf,6.131018e+07,2.973229e+07,114.617733,107.059110,10.977320,7,1.0
3,consumption,ridge,6.113812e+07,3.293630e+07,130.124149,275.651534,12.145734,7,2.0
0,consumption,lasso,6.225246e+07,3.382406e+07,132.454230,266.873424,12.474160,7,3.0
1,consumption,naive,8.178532e+07,4.732278e+07,153.493855,99.865208,17.451891,7,4.0
4,consumption,snaive,8.178532e+07,4.732278e+07,153.493855,99.865208,17.451891,7,5.0
7,price,rf,2.862114e+01,2.294529e+01,17.448944,13.716017,1.065262,7,1.0
5,price,lasso,3.049577e+01,2.646795e+01,20.635017,16.978567,1.227695,7,2.0
8,price,ridge,3.078770e+01,2.669834e+01,20.852705,17.140981,1.238011,7,3.0
6,price,naive,4.086930e+01,3.613873e+01,22.672647,24.956759,1.655895,7,4.0
9,price,snaive,4.086930e+01,3.613873e+01,22.672647,24.956759,1.655895,7,5.0


## 10) Entrenamiento final + Intervalos de predicción (Conformal) + Export

In [14]:

from joblib import dump
import pathlib

MODELS_OUT = pathlib.Path("models"); MODELS_OUT.mkdir(exist_ok=True)
RESULTS_OUT = pathlib.Path("results"); RESULTS_OUT.mkdir(exist_ok=True)

BEST_BY_TARGET = (
    summary_df.loc[~summary_df["model"].isin(["naive","snaive"])]
    .sort_values(["logical_target","sMAPE_mean"])
    .groupby("logical_target").first().reset_index()
)[["logical_target","model"]]

print("Mejores por objetivo:")
print(BEST_BY_TARGET)

def conformal_interval(residuals: np.ndarray, alpha: float = 0.1):
    q = np.quantile(np.abs(residuals), 1 - alpha)
    return float(q)

artifacts = []
for _, row in BEST_BY_TARGET.iterrows():
    logical = row["logical_target"]; y_col = {"consumption":"consumption","price":"price","profit":"profit"}[logical]
    X_all, y_all, feat_cols = build_xy(df, y_col)

    # Calibración con el último pliegue
    tr_idx, va_idx, tr_years, va_years = CV_FOLDS[-1]
    X_tr = X_all.loc[tr_idx].dropna(); y_tr = y_all.loc[X_tr.index]
    X_va = X_all.loc[va_idx].dropna(); y_va = y_all.loc[X_va.index]

    mdl = clone(model_registry()[row["model"]])
    mdl.fit(X_tr, y_tr)
    pred_va = mdl.predict(X_va)
    resid = y_va.values - pred_va
    q80 = conformal_interval(resid, alpha=0.2)
    q95 = conformal_interval(resid, alpha=0.05)

    # Entrena final con todo
    X_fit = X_all.dropna(); y_fit = y_all.loc[X_fit.index]
    mdl.fit(X_fit, y_fit)

    path_m = MODELS_OUT / f"{logical}_{row['model']}.joblib"
    dump({
        "model": mdl,
        "feat_cols": feat_cols,
        "group_cols": GROUP_COLS,
        "y_col": y_col,
        "PI80_abs": q80,
        "PI95_abs": q95
    }, path_m)

    artifacts.append((logical, row["model"], str(path_m), q80, q95))

# Guardados de métricas
results_df.to_csv(RESULTS_OUT / "cv_metrics_by_fold.csv", index=False)
summary_df.to_csv(RESULTS_OUT / "cv_summary_by_model.csv", index=False)

pd.DataFrame(artifacts, columns=["target","model","path","PI80_abs","PI95_abs"])


Mejores por objetivo:
  logical_target model
0    consumption    rf
1          price    rf
2         profit    rf


,target,model,path,PI80_abs,PI95_abs
0,consumption,rf,models/consumption_rf.joblib,7.117636e+05,1.893373e+07
1,price,rf,models/price_rf.joblib,3.935744e+01,3.935744e+01
2,profit,rf,models/profit_rf.joblib,8.022487e+08,6.777304e+09


## 11) Función de inferencia (sin gráficos)

In [ ]:

from joblib import load

def predict_with_intervals(df_new: pd.DataFrame, artifact_path: str) -> pd.DataFrame:
    art = load(artifact_path)
    mdl = art["model"]
    y_col = art["y_col"]
    feat_cols = art["feat_cols"]
    group_cols = art["group_cols"]
    q80, q95 = art["PI80_abs"], art["PI95_abs"]

    # Reconstruye features con la misma lógica
    X_new, _, _ = build_xy(df_new, y_col)
    X_new = X_new.dropna()

    preds = mdl.predict(X_new)
    base_cols = ["year"] + group_cols
    out = df_new.loc[X_new.index, base_cols].copy()
    out[f"pred_{y_col}"] = preds
    out[f"pred_{y_col}_lo80"] = preds - q80
    out[f"pred_{y_col}_hi80"] = preds + q80
    out[f"pred_{y_col}_lo95"] = preds - q95
    out[f"pred_{y_col}_hi95"] = preds + q95
    return out
